In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from io import StringIO

# Set up WebDriver with webdriver-manager
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Open a website
driver.get("https://www2.bmf.com.br/pages/portal/bmfbovespa/lumis/lum-ajustes-do-pregao-ptBR.asp")

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from io import StringIO
import pandas as pd
from selenium.webdriver.common.action_chains import ActionChains
import time

def atualizar_pagina(ele_pesquisa,data_br,ele_ok):
    # Focus on the search bar and clear it by selecting all text and pressing backspace
    actions = ActionChains(driver)
    actions.click(ele_pesquisa)  # Focus on the search bar
    actions.key_down(Keys.CONTROL).send_keys("a").key_up(Keys.CONTROL)  # Ctrl + A to select all
    actions.send_keys(Keys.BACKSPACE)  # Press Backspace to delete the selected text
    actions.send_keys(data_br)  # Press Backspace to delete the selected text
    actions.send_keys(Keys.ENTER)
    actions.click(ele_ok) 
    actions.perform()


def get_table(ele_table):
    table_data = StringIO(ele_table.get_attribute("outerHTML"))
    precos_ajustes = pd.read_html(table_data,thousands='^')[0]
    return precos_ajustes

In [ ]:
data_range = pd.date_range('2010-02-19','2025-02-24',freq='B')
for data in data_range[0:]:
    data_br = data.strftime(format='%d-%m-%Y')
    ele_pesquisa = driver.find_element(By.XPATH,'//*[@id="dData1"]')
    ele_ok = driver.find_element(By.XPATH,'//*[@id="divContainerIframeBmf"]/div[1]/div/form/div/div[2]/button')
    atualizar_pagina(ele_pesquisa,data_br,ele_ok)
    time.sleep(1)
    try:
        ele_table = driver.find_element(By.CSS_SELECTOR,'#tblDadosAjustes')
        precos_ajustes = get_table(ele_table)
        precos_ajustes['data'] = data
        precos_ajustes.to_csv(f'./tabelas/{data.date()}.csv')
    except:
        precos_ajustes = []
    time.sleep(1)
